In [1]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 933.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00


In [2]:
pip install tensorflow==2.14

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.9 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling te

In [1]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [3]:
tweets_data = load_dataset("csv", data_files="./merged_data.csv")
tweets_data = tweets_data["train"].train_test_split(test_size=0.1)
tweets_data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 315
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 35
    })
})

In [3]:
pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.7 MB/s eta 0:00:00


In [4]:
MODEL_NAME = "google/flan-t5-small"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [doc for doc in examples["input"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   # The "labels" are the tokenized outputs (use examples["output"] as the target):
   labels = tokenizer(examples["output"], max_length=512, truncation=True)
   print("printing labelsss ", labels)

   model_inputs["labels"] = labels["input_ids"]
   print("printing model", model_inputs)
   return model_inputs


In [6]:
tokenized_dataset = tweets_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/315 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


printing labelsss  {'input_ids': [[3320, 18256, 4633, 1298, 27, 317, 25, 3413, 43, 612, 3, 9, 1627, 613, 3, 11182, 748, 3, 18, 27, 373, 169, 25, 16, 82, 4062, 13, 207, 2005, 7, 5, 1], [1119, 12, 4839, 82, 223, 269, 230, 38, 34, 341, 4781, 7, 1], [3320, 17137, 969, 448, 32, 7, 15, 536, 4244, 1024, 3, 23, 3337, 233, 2258, 540, 31, 7, 30, 1990, 2452, 41, 235, 32, 4813, 14396, 42, 2087, 3, 6296, 11390, 55, 68, 3382, 62, 56, 217, 3, 102, 5, 7, 25, 2299, 1312, 16, 8, 5585, 55, 1], [3320, 667, 1123, 15, 518, 16497, 5, 5, 165, 250, 165, 1107, 45, 25, 5, 5, 10339, 256, 417, 921, 133, 241, 82, 333, 1], [3320, 7066, 9203, 29, 29, 34, 31, 7, 78, 6819, 55, 3, 23, 214, 24, 132, 31, 7, 1077, 78, 231, 72, 16, 3, 15, 26, 4662, 106, 68, 3, 23, 3, 29, 15, 15, 15, 15, 15, 15, 15, 15, 6958, 12, 805, 424, 55, 3, 40, 51, 9, 32, 1], [3320, 22517, 15, 1047, 4232, 940, 71, 210, 1986, 27, 21607, 15, 27, 3, 40, 29, 32, 3, 88, 3041, 15, 7, 3, 63, 76, 72, 2824, 9491, 7, 7, 413, 10786, 172, 58, 27, 31, 51, 3, 17, 15

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

printing labelsss  {'input_ids': [[1142, 3, 16349, 11, 410, 489, 9501, 7, 16, 209, 1781, 55, 275, 6, 10617, 47, 1095, 11, 13139, 140, 21, 6729, 5, 5, 363, 3, 9, 194, 12, 414, 8, 847, 1], [140, 15, 107, 7270, 7, 11572, 265, 469, 3, 23, 214, 256, 3, 13366, 129, 3, 4243, 11863, 44, 496, 21, 3, 31, 2264, 5109, 4890, 31, 116, 3, 23, 410, 2111, 314, 716, 3, 63, 1135, 30, 8, 2601, 87, 9719, 1603, 1], [175, 378, 25, 103, 4781, 82, 6382, 5, 5, 131, 2145, 5, 1], [3320, 21136, 11054, 2866, 4, 3, 9, 210, 210, 3, 107, 202, 29, 23, 6, 43, 3, 76, 3, 15, 19422, 135, 8665, 3, 23, 27975, 135, 81, 34, 6, 150, 8776, 3, 5, 5, 4541, 5, 5, 165, 224, 3, 9, 2210, 256, 3, 32, 32, 42, 651, 1], [325, 4164, 1771, 12574, 65, 4910, 8, 15030, 7, 62, 141, 4355, 1603, 4701, 63, 11066, 19, 3, 7, 29, 32, 32, 8128, 11, 27, 183, 3, 11907, 95, 30, 14059, 19804, 1], [27, 14371, 9, 161, 91, 68, 27, 2541, 4632, 82, 1268, 469, 5, 1], [3320, 7, 3690, 4010, 3233, 9, 1137, 82, 1237, 1312, 655, 65, 3, 5883, 920, 2400, 25, 5, 1], [5

In [4]:
pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=a7b665b99369a341edd66ecc29c98b93838354b7da010e057e700fc437f2e288
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [7]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [8]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

In [5]:
pip install "transformers[torch]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.6 MB/s eta 0:00:00


In [9]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 3
# PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 5

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results2",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   # per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False,
   optim= 'adafactor'

)

In [10]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [11]:
trainer.train()

trainer.save_model('./model_path')


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,3.785829,0.124762,0.025815,0.116922,0.117377
2,No log,3.801742,0.135476,0.029364,0.130301,0.128082
3,No log,3.846425,0.134290,0.026507,0.127467,0.128025
4,No log,3.897134,0.133143,0.023910,0.128461,0.128113
5,3.376900,3.933257,0.136150,0.023910,0.130955,0.129682


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [12]:
last_checkpoint = "./model_path"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
inputs = "Paraphrase the following tweet without any explanation before or after it: Many viewers, including critics, enjoy Come Dine With Me and there are many fans."


In [14]:
inputs = tokenizer(inputs, return_tensors="pt")
outputs = finetuned_model.generate(**inputs,max_length = 500)
answer = tokenizer.decode(outputs[0])
print(answer)

<pad> The movie is a great one, and it's a great movie. It's a great movie.</s>


In [46]:
!zip -r 'final_model.zip' './model_path'

updating: model_path/ (stored 0%)
updating: model_path/special_tokens_map.json (deflated 85%)
updating: model_path/training_args.bin (deflated 50%)
updating: model_path/config.json (deflated 62%)
updating: model_path/model.safetensors (deflated 7%)
updating: model_path/generation_config.json (deflated 29%)
updating: model_path/spiece.model (deflated 48%)
updating: model_path/added_tokens.json (deflated 83%)
updating: model_path/tokenizer_config.json (deflated 95%)


In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

# Beam Search

context = "Once, a group of frogs were roaming around the forest in search of water."
text = inputs = "Paraphrase the following tweet without any explanation before or after it: Many viewers, including critics, enjoy Come Dine With Me and there are many fans."


encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

model.eval()
beam_outputs = model.generate(
    input_ids=input_ids,attention_mask=attention_mask,
    max_length=128,
    early_stopping=True,
    num_beams=15,
    num_return_sequences=3

)

print ("\n\n")
print ("Original: ",context)
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

device  cpu


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(





Original:  Once, a group of frogs were roaming around the forest in search of water.
paraphrasedoutput: There are many followers, including critics, who enjoy Come Dine With Me, and there are many followers.
paraphrasedoutput: Many viewers, including critics, love Come Dine With Me, and there are many followers.
paraphrasedoutput: Before or after, many viewers, including critics, love Come Dine With Me, and there are many followers.
